In [13]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [5]:
# Import existing api configuration code
import sys
sys.path.append('..')

In [6]:
from config import * 

ACCOUNT STATUS IS ACTIVE


In [7]:
polygon = api.polygon

In [57]:
def fetch_dividends(sym):
    """Given a symbol returns a dataframe of dividends"""
    symbol_dividends = polygon.dividends(sym)
    fields = ['amount', 'declaredDate', 'exDate', 'indicated', 'paymentDate', 'qualified', 'recordDate', 'symbol', 'type']
    # Initialize general data structure
    output = {}
    for field in fields:
        output[field] = []
    
    for dividend in symbol_dividends:
        for field in fields:
            converted = dividend._raw
            try:
                output[field].append(converted[field])
            except:
                output[field].append(np.nan)
            
    df = pd.DataFrame.from_dict(output)
    for date_field in [x for x in fields if 'Date' in x]:
        df[date_field] = pd.to_datetime(df[date_field])
    return df 

def fetch_stock(sym, limit = 10000):
    """Fetches limit entries for a given stock symbol and puts it in a dataframe"""
    agg = polygon.historic_agg('day', sym, limit=limit).df
    dates = [str(x).split("T")[0] for x in agg.index.values]
    agg['formatted_dates'] = pd.to_datetime(dates)
    return agg

def fetch_dividend_pricing(sym, limit = 10000):
    """Returns relevant price characteristics for a dividend stock over some period of time
    Includes
    Exdate - Ex dividend date 
    amount - The amount of the dividend output
    open   - The opening price of the stock
    high   - The high price of the stock that day 
    low    - The low price of the stock that day 
    close  - The closing price of the stock that day    
    """
    a_dat = fetch_stock(sym, limit = limit)
    a_div = fetch_dividends(sym)
    joining_a_dat = a_dat[['open', 'high', 'low', 'close', 'formatted_dates']]
    joining_a_dat = joining_a_dat.reset_index().drop(['day'], axis=1).set_index('formatted_dates')
    joining_a_div = a_div.drop(['type','qualified','indicated', 'paymentDate', 'recordDate', 'symbol', 'declaredDate'], axis=1).set_index('exDate')
    joined_data = joining_a_div.join(joining_a_dat)
    return joined_data[pd.notnull(joined_data.open)]
    
    

In [43]:
appl_dat = fetch_stock("AAPL")
appl_div = fetch_dividends("AAPL")

In [58]:
appl_dat = fetch_dividend_pricing("AAPL")

In [60]:
appl_dat.head()

,amount,open,high,low,close
exDate,,,,,
2018-11-08,0.73,210.00,210.12,206.75,208.4900
2018-08-10,0.73,207.36,209.10,206.67,207.5300
2018-05-11,0.73,189.49,190.06,187.45,188.5900
2018-02-09,0.63,157.07,157.89,150.24,151.2545
2017-11-10,0.63,175.11,175.38,174.27,174.6700


In [64]:
appl_dat.index[0] - pd.Timedelta(days = 2)

Timestamp('2018-11-06 00:00:00')

In [65]:
# TODO Find a way to determine whether the stock recovers well post-dividend distribution